<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_dnn_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 sample_submission.csv	test-metadata.csv	  train-metadata.csv
logistic_regression.pkl  scaler.pkl		test-metadata.csv.gsheet  train-metadata-v1.csv
random_forest_model.pkl  test-image.hdf5	train-image.hdf5


In [4]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-4-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [5]:
train_df, test_df = train_test_split(metadata_df, test_size=0.2, random_state=42)

Y_test = test_df['target']


In [6]:
Y = train_df["target"]

dropped_columns = ["isic_id", "target", "patient_id", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
X = train_df.drop(columns=dropped_columns)

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')

X.fillna(0, inplace=True)

In [7]:
# Identify columns with mixed types

for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Identify categorical features
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# Create a ColumnTransformer with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_columns),  # One-hot encode categorical columns
        ('num', StandardScaler(), numerical_columns)  # Scale numerical columns
    ]
)

X_train_encoded = preprocessor.fit_transform(X)


# Assuming you've already preprocessed your categorical variables
model = Sequential([
    Dense(64, input_shape=(X_train_encoded.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
class_weight = {0: 1, 1: 100}  # Adjust the weight ratio to balance the classes

model.fit(X_train_encoded, Y, epochs=20, batch_size=32)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9985 - loss: 0.0134
Epoch 2/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9989 - loss: 0.0064
Epoch 3/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - accuracy: 0.9990 - loss: 0.0062
Epoch 4/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9989 - loss: 0.0057
Epoch 5/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9990 - loss: 0.0056
Epoch 6/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9990 - loss: 0.0052
Epoch 7/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9990 - loss: 0.0053
Epoch 8/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9989 - loss: 0.0055
Epoch 9/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9991 - loss: 0.0049
Epoch 10/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9991 - loss: 0.0045
Epoch 11/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9991 - loss: 0.00

In [9]:
# test_df['sex'] = test_df['sex'].replace(pd.NA, 'unknown')
# test_df['anatom_site_general'] = test_df['anatom_site_general'].replace(pd.NA, 'unknown')
# test_df['lesion_id'] = test_df['lesion_id'].replace(pd.NA, 'unknown')
# test_df['iddx_2'] = test_df['iddx_2'].replace(pd.NA, 'unknown')
# test_df['iddx_3'] = test_df['iddx_3'].replace(pd.NA, 'unknown')
# test_df['iddx_4'] = test_df['iddx_4'].replace(pd.NA, 'unknown')
# test_df['mel_mitotic_index'] = test_df['mel_mitotic_index'].replace(pd.NA, 'unknown')

# test_df.fillna(0, inplace=True)
# # Identify columns with mixed types
# for column in test_df.columns:
#   if test_df[column].apply(type).nunique() > 1:
#     print(f"{column}: {test_df[column].unique()}")


test_isic_ids = test_df['isic_id']
test_df = test_df.drop(columns=dropped_columns)
test_df.head()

,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
278442,55.0,female,lower extremity,3.15,TBP tile: close-up,3D: XP,15.998595,8.383633,28.868481,25.256029,...,1.007245,3.890118,1.672431,0.136691,140,160.508423,354.662811,72.125671,ACEMID MIA,CC-0
215021,40.0,male,posterior torso,3.03,TBP tile: close-up,3D: XP,17.651690,12.653870,31.869840,29.712320,...,0.987079,2.747920,2.701037,0.264865,150,144.663000,1240.713000,94.129270,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC
209685,85.0,male,lower extremity,2.72,TBP tile: close-up,3D: XP,11.409180,9.934399,16.911940,19.329230,...,1.088047,1.835973,2.095205,0.239726,130,-290.056300,103.316300,63.942630,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC
29648,55.0,male,posterior torso,3.58,TBP tile: close-up,3D: XP,15.982470,12.767900,22.729350,21.831640,...,0.592206,1.047462,1.364019,0.287234,135,-81.866640,1614.268000,72.240660,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC
323386,70.0,male,posterior torso,5.52,TBP tile: close-up,3D: XP,13.933450,7.950915,16.302050,17.114270,...,2.765024,2.548177,2.200729,0.460039,130,80.211790,1107.658000,153.928100,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC


In [10]:
# Make predictions on the test data
X_test_encoded = preprocessor.fit_transform(test_df)
y_pred_proba = model.predict(X_test_encoded)  # Probability of positive class (e.g., malignant)

# Create a DataFrame to map predictions back to 'isic_id'
predictions_df = pd.DataFrame({
    'isic_id': test_isic_ids,
    'target': y_pred_proba.flatten()
})


predictions_df


2507/2507 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


,isic_id,target
278442,ISIC_6973879,2.327157e-05
215021,ISIC_5407194,1.068693e-06
209685,ISIC_5273739,8.112011e-05
29648,ISIC_0802250,1.509987e-06
323386,ISIC_8084953,1.196102e-04
...,...,...
2741,ISIC_0139843,3.277821e-05
239797,ISIC_6027282,5.562295e-07
243397,ISIC_6115419,3.925266e-06
86684,ISIC_2218691,1.706644e-05


In [11]:
y_pred_proba = np.nan_to_num(y_pred_proba, nan=0)


In [12]:
y_pred_proba.max()

0.9783659

In [13]:
Y_test

,target
278442,0
215021,0
209685,0
29648,0
323386,0
...,...
2741,0
239797,0
243397,0
86684,0


In [14]:
import numpy as np
from sklearn.metrics import roc_curve, auc


# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, y_pred_proba)

# Specify the range of TPR you are interested in (e.g., above 80%)
# Filter the points where TPR is above 0.8
tpr_threshold = 0.8
valid_range = tpr >= tpr_threshold

# Compute pAUC within the valid range
partial_auc = auc(fpr[valid_range], tpr[valid_range])

print(f"Partial AUC for TPR > 80%: {partial_auc:.2f}")


Partial AUC for TPR > 80%: 0.87


In [15]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Assume you have trained your pipeline as described before

# Save the entire pipeline to a pickle file
with open('/content/drive/MyDrive/kaggle/isic-2024-data/sequential_dnn.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model pipeline saved successfully!")


Model pipeline saved successfully!
